<a href="https://colab.research.google.com/github/JulioLaz/chatbot_spring_02/blob/main/Chatbot_Inteligente_Alura_Sprint2.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# 1. Configurar ambiente

In [1]:
#Instalando bibliotecas
import pandas as pd
import re, os, random, pickle
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.metrics.pairwise import cosine_similarity
import spacy
!python -m spacy download es_core_news_md
!pip install jellyfish
import jellyfish
!pip install transformers
from transformers import BertForSequenceClassification
from transformers import BertTokenizer
import torch
!pip install python-docx
from docx import Document
import csv
import nltk
nltk.download('punkt')

#Definiendo variables del proyecto:
nlp = spacy.load('es_core_news_md')

#Conectando al Google Drive
from google.colab import drive
drive.mount('/content/drive')
folder = '/content/drive/MyDrive/Chatbot'

2023-09-25 22:26:16.348744: W tensorflow/compiler/tf2tensorrt/utils/py_utils.cc:38] TF-TRT Warning: Could not find TensorRT
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 42.3/42.3 MB 18.5 MB/s eta 0:00:00
✔ Download and installation successful
You can now load the package via spacy.load('es_core_news_md')
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.2/1.2 MB 17.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 7.6/7.6 MB 60.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 294.9/294.9 kB 33.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 7.8/7.8 MB 123.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.3/1.3 MB 99.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 5.6/5.6 MB 48.7 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
  Created wheel for python-docx: filename=python_docx-0.8.11-py3-none-any.whl size=184487 sha256=3bc95ec36b16d68ccd6db964629df8e61b37ecbe5bc73eba63bab03a40c9bd10
  Stored in d

[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Unzipping tokenizers/punkt.zip.


Mounted at /content/drive


# 2. Importar verbos

In [2]:
import pickle

archivo_pickle_verbos = "/content/drive/MyDrive/Chatbot/verbos/lista_verbos.pickle"
archivo_pickle_verbos_irregulares = "/content/drive/MyDrive/Chatbot/verbos/verbos_irregulares.pickle"

# Importar la lista_verbos:
with open(archivo_pickle_verbos, "rb") as archivo:
        lista_verbos = pickle.load(archivo)

# Importar el diccionario:
with open(archivo_pickle_verbos_irregulares, "rb") as archivo_irr:
        lista_verbos_irregulares = pickle.load(archivo_irr)


print(lista_verbos)
print('---------------------------------')
print(lista_verbos_irregulares)

['parar', 'recomendar', 'cancelar', 'fanatizar', 'amaran o amasen', 'exponer', 'obedecer', 'quejar', 'echar', 'legitimar', 'perjudicar', 'organizar', 'molar', 'objetar', 'considerar', 'golear', 'mover', 'acertar', 'reunir', 'regir', 'ilusionar', 'simpatizar', 'conjeturar', 'helar', 'quitar', 'amariamos', 'destacar', 'llegar', 'sincronizar', 'lesionar', 'seducir', 'asistir', 'conservar', 'acordar', 'salvar', 'relucir', 'graduar', 'forzar', 'dar', 'deplorar', 'batear', 'mofar', 'estropear', 'aplastar', 'wasapeo', 'gestionar', 'suprimir', 'gruñir', 'progresar', 'suscribir', 'noticiar', 'cavar', 'alejar', 'galopar', 'virar', 'medir', 'actualizar', 'humanizar', 'convivir', 'gratificar', 'digerir', 'tocar', 'zonificar', 'amariais', 'aterrizar', 'hojear', 'cometer', 'sufrir', 'reciclar', 'obturar', 'divertir', 'ondear', 'listar', 'determinar', 'alentar', 'sumar', 'reflexionar', 'ames', 'anotar', 'mitificar', 'escribir', 'ilustrar', 'obtener', 'subir', 'socorrer', 'desprender', 'agregar', 'gan

#3. Tratamiento de datos

In [21]:
#Función para encontrar la raiz de las palabras
def raiz(palabra):
    palabra_encontrada = palabra
    max_similitud = 0.0

    for verbo in lista_verbos:
        similitud = jellyfish.jaro_similarity(palabra, verbo)
        if similitud > max_similitud:
            max_similitud = similitud
            palabra_encontrada = verbo

    if max_similitud >= 0.93:
        return palabra_encontrada, max_similitud
    else:
        return palabra, max_similitud

def tratamiento_texto(texto):
  trans = str.maketrans('áéíóú','aeiou')
  texto = texto.lower()
  texto = texto.translate(trans)
  texto = re.sub(r"[^\w\s+\-*/]", '', texto)
  texto = " ".join(texto.split())
  return texto

#Función para reemplazar el final de una palabra por 'r'
def reemplazar_terminacion(frase):
    terminaciones = ["es", "me", "as", "ste", "te"]
    palabras = frase.split()

    for i, palabra in enumerate(palabras):
        for terminacion in terminaciones:
            if palabra.endswith(terminacion) and len(palabra) > len(terminacion):
                palabras[i] = palabra[:-len(terminacion)] + "r"
                break

    nueva_frase = " ".join(palabras)
    return nueva_frase

#Función para adicionar o eliminar tokens
def revisar_tokens(texto, tokens):
  #tokens: Es una lista vacia o no
  #texto: Es la pregunta del usuario
  if len(tokens)==0: #tokens está vacio, vamos a incluir tokens especiales
    #Si encuentras alguna palabra compuesta especial en el texto adiciona su token:
    #Si existe alguna: ['cientifico de datos', 'data scientist'] -> tokens.append('datascientist')
    #Si existe alguna: ['ciencia de datos', 'data science'] -> tokens.append('datascience')
    #Si existe alguna: ['elprofealejo', 'el profe alejo', 'profe alejo', 'profealejo'] -> tokens.append('elprofealejo')
    # if [name for name in ['cientifico de datos', 'data scientist','ciencia de datos', 'data science','elprofealejo', 'el profe alejo', 'profe alejo', 'profealejo']if name in texto]:
    if [name for name in ['cientifico de datos', 'data scientist'] if name in texto]: tokens.append('datascientist')
    elif [name for name in ['ciencia de datos', 'data science'] if name in texto]: tokens.append('datascience')
    elif [name for name in ['elprofealejo', 'el profe alejo', 'profe alejo', 'profealejo'] if name in texto]: tokens.append('elprofealejo')
    # print(tokens)
  else: #tokens no está vacio, vamos a eliminar tokens irrelevantes
    elementos_a_eliminar = ["cual", "que", "quien", "cuanto", "cuando", "como"]
    if 'hablame' in texto and 'hablar' in tokens: elementos_a_eliminar.append('hablar')
    elif 'cuentame' in texto and 'contar' in tokens: elementos_a_eliminar.append('contar')
    elif 'hago' in texto and 'hacer' in tokens: elementos_a_eliminar.append('hacer')
    elif 'entiendes' in texto and 'entender' in tokens: elementos_a_eliminar.append('entender')
    elif 'sabes' in texto and 'saber' in tokens: elementos_a_eliminar.append('saber')
    #Ahora que tenemos nuestra blacklist(tokens irrelevantes) elimínalos de nuestra lista tokens
    tokens = [token for token in tokens if token not in elementos_a_eliminar]
    # print(tokens)
  return tokens

#Función para devolver los tokens normalizados del texto
# def normalizar(texto):
#   tokens=[]
#   tokens=revisar_tokens(tratamiento_texto(texto), tokens)
#   doc = nlp(texto)
#   for t in doc:
#     lemma=lista_verbos_irregulares.get(t.text, t.lemma_.split()[0])
#     lemma=re.sub(r'[^\w\s+\-*/]', '', lemma)
#     if t.pos_ in ('VERB','PROPN','PRON','NOUN','AUX','SCONJ','ADJ','ADV','NUM') or lemma in lista_verbos:
#       if t.pos_=='VERB':
#         lemma = reemplazar_terminacion(lemma)
#         tokens.append(raiz(tratamiento_texto(lemma)))
#       else:
#         tokens.append(tratamiento_texto(lemma))

#   tokens = list(dict.fromkeys(tokens))
#   tokens = list(filter(None, tokens))
#   tokens = revisar_tokens(tratamiento_texto(texto), tokens)
#   return str(tokens)

def normalizar(texto):
    tokens = []
    tokens = revisar_tokens(tratamiento_texto(texto), tokens)
    doc = nlp(texto)
    for t in doc:
        # Obtener el lemma
        lemma = lista_verbos_irregulares.get(t.text, t.lemma_)

        # Verificar si lemma es una cadena no vacía
        if lemma and isinstance(lemma, str):
            lemma = re.sub(r'[^\w\s+\-*/]', '', lemma)

            if t.pos_ in ('VERB','PROPN','PRON','NOUN','AUX','SCONJ','ADJ','ADV','NUM') or lemma in lista_verbos:
                if t.pos_ == 'VERB':
                    lemma = reemplazar_terminacion(lemma)
                    tokens.append(raiz(tratamiento_texto(lemma)))
                else:
                    tokens.append(tratamiento_texto(lemma))

    tokens = list(dict.fromkeys(tokens))
    tokens = list(filter(None, tokens))
    tokens = revisar_tokens(tratamiento_texto(texto), tokens)
    tokens_str = str(tokens)
    return tokens_str


# 4. Cargar bases de documentos

In [22]:
#Importando bases de dialogo fluído
txt_folder_path = folder+'/dialogos'
lista_documentos=[x for x in os.listdir(txt_folder_path) if x.endswith(".txt")]
lista_dialogos, lista_dialogos_respuesta, lista_tipo_dialogo = [],[],[]
for idx in range(len(lista_documentos)):
  f=open(txt_folder_path+'/'+lista_documentos[idx], 'r', encoding='utf-8', errors='ignore')
  estado = True
  for line in f.read().split('\n'):
    if estado:
      line_tratado = tratamiento_texto(line)
      lista_dialogos.append(line_tratado)
      lista_tipo_dialogo.append(lista_documentos[idx][:-4])
    else:
      lista_dialogos_respuesta.append(line)
    estado=not estado

#Creando Dataframe de diálogos
datos = {'dialogo':lista_dialogos,'respuesta':lista_dialogos_respuesta,'tipo':lista_tipo_dialogo,'interseccion':0,'jaro_winkler':0,'probabilidad':0}
df_dialogo = pd.DataFrame(datos)
df_dialogo = df_dialogo.drop_duplicates(keep='first')
df_dialogo.reset_index(drop=True, inplace=True)

#Importando bases csv
txt_folder_path = folder+'/documentos'
lista_documentos=[x for x in os.listdir(txt_folder_path) if x.endswith(".csv")]
documento_csv = ''
for idx in range(len(lista_documentos)):
  with open(txt_folder_path+'/'+lista_documentos[idx], "r", encoding="utf-8") as archivo_csv:
    lector_csv = csv.reader(archivo_csv)
    for fila in lector_csv:
      if fila[-1]!='frase':
        documento_csv += fila[-1]

#Importando bases docx
lista_documentos=[x for x in os.listdir(txt_folder_path) if x.endswith(".docx")]
documento_docx = ''
for idx in range(len(lista_documentos)):
  for paragraph in Document(txt_folder_path+'/'+lista_documentos[idx]).paragraphs:
    documento_docx += paragraph.text.replace('*','\n\n*').replace('-','\n-')

#Importando bases txt
lista_documentos=[x for x in os.listdir(txt_folder_path) if x.endswith(".txt")]
documento_txt = ''
for idx in range(len(lista_documentos)):
  with open(txt_folder_path+'/'+lista_documentos[idx], "r", encoding="utf-8") as archivo_txt:
    lector_txt = archivo_txt.read()
    for fila in lector_txt:
      documento_txt += fila

documento = documento_csv + documento_txt + documento_docx
lista_frases = nltk.sent_tokenize(documento,'spanish')
lista_frases_normalizadas = [' '.join(normalizar(x)) for x in lista_frases]

###Guardar DF en drive:

In [23]:
import pandas as pd
import os

def guardar_dataframe(dataframe):
  carpeta = "/content/drive/MyDrive/Chatbot"
  nombre_archivo = "df_dialogo_spring_02.csv"

  if not os.path.exists(carpeta):
    os.makedirs(carpeta)
  archivo_csv = os.path.join(carpeta, nombre_archivo)
  dataframe.to_csv(archivo_csv, index=False)

# 5. Buscar respuesta del Chatbot

In [27]:
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.metrics.pairwise import cosine_similarity
import jellyfish

def dialogo(user_response):
    # Tratamiento de texto
    user_response = tratamiento_texto(user_response)  # Tratando el texto
    user_response = re.sub(r"[^\w\s]", '', user_response)  # Elimina signos de puntuación

    df = df_dialogo.copy()

    tfidf_vectorizer = TfidfVectorizer()

    for idx, row in df.iterrows():
        # Calcular la similitud de coseno usando TF-IDF
        tfidf_matrix = tfidf_vectorizer.fit_transform([user_response, row['dialogo']])
        cos_sim = cosine_similarity(tfidf_matrix[0], tfidf_matrix[1])

        df.at[idx, 'interseccion'] = len(set(user_response.split()) & set(row['dialogo'].split()))/len(user_response.split())
        df.at[idx, 'similarity'] = cos_sim[0][0]
        df.at[idx, 'jaro_winkler'] = jellyfish.jaro_winkler_similarity(user_response, row['dialogo'])

        # Calcular la probabilidad:
        df.at[idx, 'probabilidad'] = max(df.at[idx, 'interseccion'], df.at[idx, 'similarity'], df.at[idx, 'jaro_winkler'])

    # Ordenar el DataFrame por probabilidad y Jaro-Winkler
    df.sort_values(by=['probabilidad', 'jaro_winkler'], inplace=True, ascending=False)

    # Obtener la probabilidad máxima
    probabilidad = round(df['probabilidad'].head(1).values[0],2)

    #Guardar df:
    guardar_dataframe(df)

    if probabilidad >= 0.93:
        print('Respuesta encontrada por el método de comparación de textos - Probabilidad: ', probabilidad)
        respuesta = df['respuesta'].head(1).values[0]
    else:
        respuesta = ''
    return respuesta

#Cargar tu modelo entrenado aqui(recuerda siempre cargar el modelo y el vectorizer o tokenizer usado en el entrenamiento del modelo):
ruta_modelo = '/content/drive/MyDrive/Chatbot/'
Modelo_TF = BertForSequenceClassification.from_pretrained(ruta_modelo)
tokenizer_TF = BertTokenizer.from_pretrained(ruta_modelo)

#Función para dialogar utilizando el modelo de Machine Learning
def clasificacion_modelo(pregunta):
  frase = normalizar(pregunta)
  frase = ' '.join(str(elemento) for elemento in frase)
  tokens = tokenizer_TF.encode_plus(
      frase,
      add_special_tokens=True,
      max_length=128,
      padding='max_length',
      truncation=True,
      return_tensors='pt'
  )
  input_ids = tokens['input_ids']
  attention_mask = tokens['attention_mask']

  with torch.no_grad():
      outputs = Modelo_TF(input_ids, attention_mask)

  etiquetas_predichas = torch.argmax(outputs.logits, dim=1)
  etiquetas_decodificadas = etiquetas_predichas.tolist()

  diccionario = {3: 'Continuacion', 10: 'Nombre', 2: 'Contacto', 13: 'Saludos', 14: 'Sentimiento', 9: 'Identidad', 15: 'Usuario', 6: 'ElProfeAlejo', 1: 'Aprendizaje', 0: 'Agradecimiento', 5: 'Edad', 4: 'Despedida', 11: 'Origen', 12: 'Otros', 7: 'Error', 8: 'Funcion'}
  llave_buscada = etiquetas_decodificadas[0]
  clase_encontrada = diccionario[llave_buscada]

  #Buscar respuesta más parecida en la clase encontrada
  df = df_dialogo[df_dialogo['tipo'] == clase_encontrada]
  df.reset_index(inplace=True)
  vectorizer = TfidfVectorizer()
  dialogos_num = vectorizer.fit_transform(df['dialogo'])
  pregunta_num = vectorizer.transform([tratamiento_texto(pregunta)])
  similarity_scores = cosine_similarity(dialogos_num, pregunta_num)
  indice_pregunta_proxima = similarity_scores.argmax()

  if max(similarity_scores)>0.5 and clase_encontrada not in ['Otros']:
    print('Respuesta encontrada por el modelo Transformers - tipo:',clase_encontrada)
    respuesta = df['respuesta'][indice_pregunta_proxima]
  else:
    respuesta = ''
  return respuesta

#Función para devolver la respuesta de los documentos
def respuesta_documento(pregunta):
  pregunta = normalizar(pregunta)
  def contar_coincidencias(frase):
    return sum(1 for elemento in pregunta if elemento in frase)

  diccionario = {valor: posicion for posicion, valor in enumerate(lista_frases_normalizadas)}
  lista = sorted(list(diccionario.keys()), key=contar_coincidencias, reverse=True)[:6]
  #Hasta aqui ya tengo mi lista con las 6 respuestas con mayor coincidencia de tokens
  #Convierte la pregunta en frase
  #Adiciona la frase convertida al final de la lista
  #Inicializa un TfidfVectorizer utilizando la función normalizar como tokenizer
  #Entrenalo con fit_transform sobre la lista y asigna el resultado a la variable tfidf
  if 'curso' not in pregunta: lista = [frase for frase in lista if 'curso' not in frase]
  lista.append(' '.join(pregunta))
  TfidfVec = TfidfVectorizer(tokenizer=normalizar)
  tfidf = TfidfVec.fit_transform(lista)
  #Aplica cosine_similarity entre el último elemento de tfidf y todos los elementos de tfidf y guárdalo en vals
  vals = cosine_similarity(tfidf[-1], tfidf)
  idx = vals.argsort()[0][-2]
  flat = vals.flatten()
  flat.sort()
  req_tfidf = round(flat[-2],2)
  if req_tfidf>=0.22:
    print('Respuesta encontrada por el método TfidfVectorizer - Probabilidad:', req_tfidf)
    respuesta = lista_frases[diccionario[lista[idx]]]
  else:
    respuesta = ''
  return respuesta

#Función para devolver una respuesta final buscada en todos los métodos disponibles
def respuesta_chatbot(pregunta):
  respuesta = dialogo(pregunta)
  if respuesta != '':
    return respuesta
  else:
    respuesta = clasificacion_modelo(pregunta)
    if respuesta != '':
      return respuesta
    else:
      respuesta = respuesta_documento(pregunta)
      if respuesta != '':
        return respuesta
      else:
        return 'Respuesta no encontrada'

# 6. Ejecutar Chatbot

In [29]:
pregunta='cursos ?'
respuesta = respuesta_chatbot(pregunta)
print(respuesta)

Respuesta encontrada por el método TfidfVectorizer - Probabilidad: 0.95
El curso 'Fundamentos de Ciencia de Datos' tiene un tiempo de duración de 8 semanas, por un precio o costo de $500 USD y con fecha de inicio en 1 de julio de 2023.


/usr/local/lib/python3.10/dist-packages/sklearn/feature_extraction/text.py:528: UserWarning: The parameter 'token_pattern' will not be used since 'tokenizer' is not None'
  warnings.warn(
